In [181]:
import pandas as pd
import numpy as np

In [182]:
fileName = 'output_audio_to_text_timestamp8_SM_P7-18_corrected.xlsx'
xls = pd.ExcelFile(fileName)
sheet_to_df_map = {}
for sheet_name in xls.sheet_names:
    sheet_to_df_map[sheet_name] = pd.read_excel(fileName, sheet_name=sheet_name)
    # you can also use sheet_index [0,1,2..] instead of sheet name.
sorted(xls.sheet_names)

['transcribed_speech_P10.wav',
 'transcribed_speech_P11.wav',
 'transcribed_speech_P12.wav',
 'transcribed_speech_P13.wav',
 'transcribed_speech_P14.wav',
 'transcribed_speech_P15.wav',
 'transcribed_speech_P16.wav',
 'transcribed_speech_P17_1.wav',
 'transcribed_speech_P17_2.wav',
 'transcribed_speech_P18.wav',
 'transcribed_speech_P1_1.wav',
 'transcribed_speech_P1_2from.wav',
 'transcribed_speech_P2.wav',
 'transcribed_speech_P3.wav',
 'transcribed_speech_P4_1.wav',
 'transcribed_speech_P4_2.wav',
 'transcribed_speech_P5_1.wav',
 'transcribed_speech_P5_2.wav',
 'transcribed_speech_P6.wav',
 'transcribed_speech_P7.wav',
 'transcribed_speech_P8.wav',
 'transcribed_speech_P9_1.wav',
 'transcribed_speech_P9_2.wav']

## Think-aloud data identification

In [183]:
# participants = ['P1','P2']
# participants = ['P1','P2','P3']
# participants = ['P3']
# participants = ['P13','P14','P15','P16','P17','P18']
# participants = ['P17','P18']
# participants = ['P13','P14','P15']

# participants = ['P5','P6','P11']

participants = ['P7']
# participants = ['P7','P8','P9','P10','P11','P12','P13','P14','P15','P17','P18']

ll = []
for participant in participants:
    counter = 0
    for i in sorted(xls.sheet_names):
        a = i.replace('.','_').split('_')[2]
        if participant == a:
            counter +=1
            sheet_to_df_map[i]['Participant'] = participant
            sheet_to_df_map[i]['Video Seq'] = counter
            ll.append(sheet_to_df_map[i])
df_f = pd.concat(ll).reset_index(drop=True)
df_f.head()

,Unnamed: 0,confidence,end,start,word,Element,Time in Tobii video,Does the car stop?,Interest Level,Duration,Attention,Distraction,Order,Video Num,Participant,Video Seq
0,0,1.000000,4.68,4.26,parking,In a parking lot,4.68,NaN,NaN,1.0,1.0,NaN,1.0,1.0,P7,1
1,1,1.000000,5.43,4.68,lots,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,P7,1
2,2,0.628816,9.72,9.48,oh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,P7,1
3,3,1.000000,11.91,11.70,and,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,P7,1
4,4,1.000000,12.15,11.94,to,to go left,12.15,NaN,NaN,1.0,1.0,NaN,1.0,1.0,P7,1


In [184]:
class think_aloud_element:
    def __init__(self, dict) -> None:
        self.start_time = dict['start']
        self.end_time = dict['end']
        self.is_distraction = dict['dist']
        self.interest_level = dict['int-level']
        self.content = dict['Element']
        self.participant = dict['Participant']
        # participant see video order
        self.order = dict['Order']
        # ordinal sequence of videos
        self.videoNum = dict['Video Num']
        # correspondance to files
        self.videoSeq = dict['Video Seq']

    def __repr__(self):
        ''' Returns a string describing this instance '''
        return "{} from {:.2f} sec to {:.2f} sec, interest_level is {:.2f}, considered {} distraction, from participant {}".format(
           self.content, self.start_time, self.end_time, self.interest_level, self.is_distraction, self.participant)

In [185]:
start = None
end = None
last_row = None
all_think_aloud = []
all_ta_cl = []
finished = False
def make_a_element(start, end):
    if (start is None) or (end is None):
        print('invalid start / end')
        return
    d = {}
    if start['Attention']==1:
        d['dist'] = False
    else:
        d['dist'] = True
    d['start'] = start['start']
    d['int-level'] = start['Interest Level']
    if pd.isnull(start['Interest Level']):
        d['int-level'] = 0
    d['end'] = end['end']
    d['Element'] = start['Element']
    d['Participant'] = start['Participant']
    d['Order'] = start['Order']
    d['Video Num'] = start['Video Num']
    d['Video Seq'] = start['Video Seq']
    all_think_aloud.append(d)
    all_ta_cl.append(think_aloud_element(d))
    # return
for index, row in df_f.iterrows():
    if pd.notnull(row['Element']) and row['Element']!='NULL':
        if end is None:
            end = last_row
            make_a_element(start, end)
        start = row
        end = None
        # finished = False
    if pd.isnull(row['Duration']) and (end is None):
        end = last_row
        make_a_element(start, end)
        # end = None
        # finished = True
    last_row = row
if end is None:
    end = last_row
    make_a_element(start, end)
start = row
end = None

invalid start / end


## Eye gaze data cleaning

In [186]:
d_eyeGaze_subset = {
    'P1':[
        [
            [7,126],
            [138,255],
            [267,386],
            [402,522],
            [657,782]
        ],
        [
            [4,98],
            [106,225],
            [234,353],
            [361,465],
            [482,600]
        ]
    ],
    'P2':[
        [
            [4,124],
            [137,256],
            [263,380],
            [392,512],
            [520,639],
            [652,777]
        ]
    ],
    'P3':[
        [
            [8,127],
            [468,563],
            [570,690],
            [696,817],
            [820,934],
            [941,1066],
            [1073,1193],
            [1199,1316],
            [1320,1440],
            [1454,1583]
        ]
    ],
    'P4':[
        [
            [1,119],
            [127,221],
            [229,350],
            [359,479],
            [484,597],
            [605,729],
            [740,861],
            [877,993],
            [1002,1120]
        ],
        [
            [0,119]
        ]
    ],
    'P13':[
        [
            [3,124],
            [143,234],
            [248,367],
            [375,493],
            [508,623],
            [632,757],
            [767,887],
            [894,1010],
            [1018,1138],
            [1148,1264]
        ]
    ],
    'P14':[
        [
            [3,119],
            [128,247],
            [257,377],
            [383,504],
            [514,639],
            [650,744],
            [750,870]
        ]
    ],
    'P15':[
        [
            [1,120],
            [130,247],
            [253,369],
            [378,498],
            [506,626],
            [633,757],
            [766,888]
        ]
    ],
    'P17':[
        [
            [0,116],
            [499,619],
            [623,743],
            [752,871]
        ],
        [
            [6,132],
            [141,236],
            [244,364],
            [372,492],
            [497,611],
            [620,737]
        ]
    ],
    'P18':[
        [
            [1,121],
            [125,244],
            [252,367],
            [374,493],
            [501,621]
        ]
    ],
    'P8':[
        [
            [1,117],
            [125,244],
            [258,378],
            [391,510],
            [518,644],
            [650,745],
            [752,872]
        ]
    ],
    'P9':[
        [
            [0,120]
        ],
        [
            [1,119],
            [126,242],
            [274,394],
            [400,520]
        ]
    ],
    'P10':[
        [
            [1,118],
            [125,244],
            [254,375],
            [379,499],
            [506,630],
            [635,729],
            [743,862],
            [869,988]
        ]
    ],
    'P11':[
        [
            [0,117],
            [124,244],
            [250,370],
            [386,506],
            [513,638],
            [643,738],
            [743,862],
            [872,992],
            [999,1113],
            [1117,1234]
        ]
    ],
    'P12':[
        [
            [2,121],
            [131,248],
            [256,373],
            [380,500],
            [504,624],
            [631,756],
            [766,884],
            [889,1004],
            [1015,1135],
            [1144,1237]   
        ]
    ],
    'P5':[
        [
            [0,117],
            [124,244],
            [256,375],
            [385,505],
            [512,637],
            [646,741],
            [750,870]
        ],
        [
            [0,119],
            [131,246],
            [252,315]
        ]
    ],
    'P6':[
        [
            [0,144],
            [155,273],
            [281,398],
            [406,526],
            [536,656],
            [666,792]
        ]
    ],
    'P11':[
        [
            [0,116],
            [123,243],
            [250,370],
            [386,505],
            [513,639],
            [643,738],
            [742,862],
            [872,993],
            [999,1114],
            [1117,1234]
        ]
    ],
    'P7':[
        [
            [1,120],
            [127,222],
            [230,350],
            [353,474],
            [478,593],
            [597,722],
            [728,848],
            [876,997],
            [1002,1124],
            [1129,1246]
        ]
    ]
}

df_eyes_raw = []
for participant in participants:
    df = pd.read_excel('./Eye Tracker Data Raw/Gazedata_'+participant+'.xlsx')
    df_eyes_raw.append(df)
    print(participant,'raw read done')

P7 raw read done


In [187]:
def preprocess(idx, df):
    df2 = df.copy()
    df2['Recording timestamp'] = (df['Recording timestamp']-df['Recording timestamp'][0])/np.timedelta64(1, 's')
    p = participants[idx]
    tt = d_eyeGaze_subset[p]
    names = sorted(df2['Participant name'].unique())
    all_subs = []
    for i, sec in enumerate(tt):
        sub = df2[df2['Participant name']==names[i]]
        all_subsets = []
        for s in sec:
            mask = (sub['Recording timestamp'] > s[0]) & (sub['Recording timestamp'] <= s[1])
            ddf = sub.loc[mask]
            all_subsets.append(ddf)
        all_subs.append(pd.concat(all_subsets))
    return pd.concat(all_subs).reset_index().drop(['index'],axis=1)

df_eyes_processed = []
for idx, df in enumerate(df_eyes_raw):
    df2 = df.copy()
    df2['Recording timestamp'] = pd.to_datetime(df2['Recording timestamp']/1000000, unit='s')
    df2['Computer timestamp'] = pd.to_datetime(df2['Computer timestamp'], unit='s', errors='coerce')
    df2 = df2.drop(['Project name', 'Export date', 'Recording name', 'Recording date',
    'Recording date UTC', 'Recording start time','Recording start time UTC', 'Recording duration', 
    'Recording Fixation filter name', 'Event', 'Recording media name', 'Recording media width', 
    'Recording media height', 'Ungrouped', 'Validity left', 'Validity right'], axis=1)
    df_eyes_processed.append(preprocess(idx, df2))
    print('participant',participants[idx],'preprocess done')

participant P7 preprocess done


In [188]:
for d in df_eyes_processed:
    d['interest_level'] = None
    d['order'] = None
    d['videoNum'] = None
    d['videoSeq'] = None
    d['is_distraction'] = None

for da in all_ta_cl:
    # participant sequence
    p = participants.index(da.participant)
    subdata = df_eyes_processed[p]
    names = sorted(subdata['Participant name'].unique())
    mask = (subdata['Recording timestamp']>da.start_time-0.025) & (subdata['Recording timestamp']<=da.start_time) & (subdata['Participant name']==names[da.videoSeq-1])
    subdata.loc[mask,'interest_level'] = da.interest_level
    subdata.loc[mask,'order'] = da.order
    subdata.loc[mask,'videoNum'] = da.videoNum
    subdata.loc[mask,'videoSeq'] = da.videoSeq
    subdata.loc[mask,'is_distraction'] = da.is_distraction
    df_eyes_processed[p] = subdata

## Output data

In [189]:
def print_summary_by_trails(p, df):
    print('For participant',p,'/n')
    print(df.groupby('videoNum').size())

### For Eye trackers

In [190]:
for df, p in zip(df_eyes_processed, participants):
    df = df[df['Sensor']=='Eye Tracker'].drop(['Sensor','Computer timestamp','Participant name'], axis=1)
    if 'videoNum' in df.columns:
        df[['videoSeq','videoNum','order']] = df[['videoSeq','videoNum','order']].fillna(method='ffill').fillna(method='bfill')
    else:
        df[['videoSeq','videoNum','order']] = df[['videoSeq','videoNum','order']].fillna(method='ffill').fillna(method='bfill')
    df.dropna(axis=1, how='all').reset_index(drop=True).to_csv('./Eye Tracker Data Processed/Eye tracking/'+p+'.csv')
    print_summary_by_trails(p, df.dropna(axis=1, how='all').reset_index(drop=True))

For participant P7 /n
videoNum
1.0     6087
2.0     4926
3.0     5885
4.0     6070
5.0     5562
6.0     6319
7.0     5842
8.0     6219
9.0     6210
10.0    5562
dtype: int64


### Acelerometer

In [191]:
for df, p in zip(df_eyes_processed, participants):
    df = df[df['Sensor']=='Accelerometer'].drop(['Sensor','Computer timestamp','Participant name'], axis=1)
    df[['videoSeq','videoNum','order']] = df[['videoSeq','videoNum','order']].fillna(method='ffill').fillna(method='bfill')
    df.dropna(axis=1, how='all').reset_index(drop=True).to_csv('./Eye Tracker Data Processed/Acelerometer/'+p+'.csv')

### Gyroscope

In [192]:
for df, p in zip(df_eyes_processed, participants):
    df = df[df['Sensor']=='Gyroscope'].drop(['Sensor','Computer timestamp','Participant name'], axis=1)
    df[['videoSeq','videoNum','order']] = df[['videoSeq','videoNum','order']].fillna(method='ffill').fillna(method='bfill')
    df.dropna(axis=1, how='all').reset_index(drop=True).to_csv('./Eye Tracker Data Processed/Gyroscope/'+p+'.csv')